In [1]:
import Pkg; Pkg.add("StatsBase")
import Pkg; Pkg.add("DataFrames")
import Pkg; Pkg.add("StatsModels")
import Pkg; Pkg.add("Logging")
import Pkg; Pkg.add("DataFramesMeta")

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


In [ ]:

using Printf,Plots, Dates, CSV, DelimitedFiles, StatsBase,ForwardDiff, StatsModels, DataFrames, Statistics, LinearAlgebra, Optim, NLsolve, FiniteDiff, ShiftedArrays, Logging, DataFramesMeta

df = readdlm("data_f.csv",',',skipstart=1) 
country = data[:,1]
date = data[:,2]
price_eur_m_whe = data[:,3]
gwh = data[:,4]
cp_meur_nsa_b1g = data[:,5]
cp_meur_nsa_d1 = data[:,6]
cp_meur_nsa_d11 = data[:,7]
cp_meur_nsa_p6 = data[:,8]
cp_meur_nsa_p7 = data[:,9]
ths_hw_b_e_nsa_emp_dc = data[:,10]
ths_hw_c_nsa_emp_dc = data[:,11]
ths_hw_j_nsa_emp_dc = data[:,12]
ths_hw_g_i_nsa_emp_dc = data[:,13]
ths_hw_total_nsa_emp_dc = data[:,14]
avg_nac_usd = data[:,15]
imp_e7000_gwh = data[:,16]
exp_e7000_gwh = data[:,17]



lagprice = price_eur_m_whe .- lag(price_eur_m_whe,1)
lagprice


loggwh = log.(gwh)
logths_hw_b_e_nsa_emp_dc = log.(ths_hw_b_e_nsa_emp_dc)
logths_hw_c_nsa_emp_dc = log.(ths_hw_c_nsa_emp_dc)
logths_hw_j_nsa_emp_dc = log.(ths_hw_j_nsa_emp_dc)
logths_hw_g_i_nsa_emp_dc = log.(ths_hw_g_i_nsa_emp_dc)
logths_hw_total_nsa_emp_dc = log.(ths_hw_total_nsa_emp_dc)
logexp_e7000_gwh = log.(exp_e7000_gwh)

country


In [ ]:

function convert_to_date(date)
    year, quarter = split(date)
end

dateN = convert_to_date.(date)
df


In [ ]:
p = plot(legend = :outertopright)

for country in unique(df[country])
    country_df = df[df[:, country] .== country]
    plot!(p, country_df[dateN], country_df[:, :price_eur_m_whe], label = country)
end

display(p)


In [ ]:
y = [dateN price_eur_m_whe]
x = [lagprice loggwh logths_hw_b_e_nsa_emp_dc logths_hw_c_nsa_emp_dc logths_hw_j_nsa_emp_dc logths_hw_g_i_nsa_emp_dc logths_hw_total_nsa_emp_dc logexp_e7000_gwh]
N = size(y,1) #population
T = 11
T


In [ ]:
function PanelOls(y,x,m=1,vvM=[])

  (T,N) = (size(y,1),size(y,2))
  K     = size(x,2)  #TxKxN
    
    vvM = trues(T,N)   #handling of NaN/missing
  Nobs = sum(vvM,dims=22)


  xx  = zeros(K,K)                     #Sum[x(t)*x(t)',t=1:T]
  xy  = zeros(K,1)                     #Sum[x(t)*y(t),t=1:T]
  for t = 1:T                             #loop over time
    y_t = y[t,:]                          #dependent variable, Nx1
    x_t = x[t,:,:]'                       #regressors, NxK
    xx .= xx .+ x_t'x_t/N
    xy .= xy .+ x_t'y_t/N
  end

    xx    = xx/T
  xy    = xy/T
  theta = xx\xy                           #ols estimates, solves xx*theta = xy

  yhat = fill(NaN,(T,N))                  #fitted values
  for i = 1:N                             #loop over cross section
    yhat[:,i] = x[:,:,i]*theta
  end
  r = y - yhat                            #fitted residuals

  h = fill(NaN,T,K,N)                     #moment conditions for individual
  for i = 1:N
    h[:,:,i]   = r[:,i].*x[:,:,i]/N
  end
  h_tM = dropdims(sum(h,dims=3),dims=3)   #TxK, moment conditions, aggregated over all

  hG_tM = fill(NaN,T,K,G)                 #moment conditions, aggregated to clusters
  for g = 1:G                             #loop over clusters
    vvg          = vvClust[:,g]
    hG_tM[:,:,g] = sum(h[:,:,vvg],dims=3)
  end
    
    s2 = sum(abs2,r)/N^2

  (omega0DK,omega0W,omega0C) = [zeros(K,K) for i=1:3]       #DK lag 0,White's,cluster
  (omega1DK,omega1W,omega1C) = [zeros(K,K,1) for i=1:3]     #lags 1 to m
  for t = 1:T                                               #loop over time
    h_it = h[t,:,:]                                         #moment conditions, KxN
    h_t  = h_tM[t,:]                                        #Kx1
    omega0DK .= omega0DK .+ h_t*h_t'                        #Kx1 * 1xK
    omega0W  .= omega0W  .+ h_it*h_it'                      #KxN * NxK
    for g = 1:G                                             #loop over clusters
      omega0C .= omega0C + hG_tM[t,:,g]*hG_tM[t,:,g]'       #Kx1 * 1xK
    end
    for j = 1:min(t-1,m)                                    #0,1,2,2,... for m=2
      omegajDK[:,:,j] = omegajDK[:,:,j] + h_t*h_tM[t-j,:]'  #h(t)*h(t-j)'
      omegajW[:,:,j]  = omegajW[:,:,j]  + h_it*h[t-j,:,:]'
      for g = 1:G                                           #loop over clusters
        omegajC[:,:,j] = omegajC[:,:,j] + hG_tM[t,:,g]*hG_tM[t-j,:,g]'   #Kx1 * 1xK
      end
    end
  end

    
    ShatDK = NWCovPs(omega0DK,omegajDK,T)          #estimate of S, DK
  ShatC  = NWCovPs(omega0C, omegajC, T)          #estimate of S, cluster
  ShatW  = NWCovPs(omega0W, omegajW, T)          #estimate of S, White's
  s2     = s2/T^2

  xx_1  = inv(xx)
  CovDK = xx_1 * ShatDK  * xx_1'                  #covariance matrix, DK
  CovC  = xx_1 * ShatC   * xx_1'                  #covariance matrix, cluster
  CovW  = xx_1 * ShatW   * xx_1'                  #covariance matrix, White's
  CovLS = xx_1 * s2                               #covariance matrix, LS

 R2  = cor(vec(y[vvM]),vec(yhat[vvM]))^2
any(.!vvM) && (yhat[.!vvM] .= NaN)              #0 -> NaN

  fnOutput = (;theta,CovDK,CovC,CovW,CovLS,R2,yhat,Nobs)

  return fnOutput
    
    
end
   
results = PanelOls.(y, x,m=1) # replace y, x, m, clust, vvM with your actual input
println("Theta: ", results.theta)
println("CovDK: ", results.CovDK)
println("CovC: ", results.CovC)
println("CovW: ", results.CovW)
println("CovLS: ", results.CovLS)
println("R2: ", results.R2)
println("yhat: ", results.yhat)
println("Nobs: ", results.Nobs)

In [ ]:
function NWCovPs(omega0,omegaj,T)
  m = size(omegaj,3)
  Shat = omega0/T^2
  for j = 1:m
    Shat .= Shat .+ (1-j/(m+1))*(omegaj[:,:,j] + omegaj[:,:,j]')/T^2
  end
  return Shat
end



In [ ]:
function FindNNPanel(y,x)
  (T,K,N) = size(x)
  vv = falses(T,N)
  for i = 1:N
    vv[:,i] = FindNNPs(view(y,:,i),view(x,:,:,i))
  end
  return vv
end

In [ ]:
function replaceNaNinYX!(Y,X)

  N = size(Y,2)

  vvM = FindNNPanel(Y,X)              #TxN, rows that have no NaNs/missings

  for i = 1:N                         #loop over cross-section
    vvi         = .!vvM[:,i]          #rows that should be set to 0.0
    Y[vvi,i]   .= 0.0
    X[vvi,:,i] .= 0.0
  end

  return vvM

end

In [ ]:
#Fix effect
function FixedEffects(y0,x0,FEType=:id)

  (T,N) = (size(y0,1),size(y0,2))
  K     = size(x0,2)            #TxKxN

  vvt = falses(T,N)
  for i = 1:N
    vvt[:,i] = FindNNPs(view(y0,:,i),view(x0,:,:,i))    #use (t,i) data or not
  end

  if in(FEType,[:id,:idt])                            #individual fixed effects
    (yAvg_i,xAvg_i) = (fill(NaN,1,N),fill(NaN,1,K,N))
    for i = 1:N              #loop over cross section
      vv            = vvt[:,i]
      yAvg_i[1,i]   = mean(view(y0,vv,i))
      xAvg_i[1,:,i] = mean(view(x0,vv,:,i),dims=1)
    end
  end

  if in(FEType,[:idt,:t])                              #time fixed effects
    (yAvg_t,xAvg_t) = (fill(NaN,T),fill(NaN,T,K))
    for t = 1:T              #loop over time
      vv          = vvt[t,:]
      yAvg_t[t]   = mean(view(y0,t,vv))
      xAvg_t[t,:] = mean(view(x0,t,:,vv),dims=2)
    end
    yAvg = mean(y0[vvt])                                  #grand mean of y
    xAvg = permutedims([mean(x0[:,k,:][vvt]) for k=1:K])  #grand means of each of x
  end

  if FEType == :id
    y = y0 .- yAvg_i                     #subtract fixed individual effects
    x = x0 .- xAvg_i
    (yAvg_t,xAvg_t,yAvg,xAvg,N_t) = (NaN,NaN,NaN,NaN,NaN)
  elseif FEType == :idt
    y = y0 .- yAvg_i .- yAvg_t .+ yAvg   #subtract individual and time fixed effects
    x = x0 .- xAvg_i .- xAvg_t .+ xAvg
  elseif FEType == :t
    y = y0 .- yAvg_t                     #subtract time fixed effects
    x = x0 .- xAvg_t
    (yAvg_i,xAvg_i) = (NaN,NaN)
  end

  yxAvg = (;yAvg_i,xAvg_i,yAvg_t,xAvg_t,yAvg,xAvg,vvt)

  return y, x, yxAvg

end

